## Test 5 weights

In [8]:
!python test.py --data data/DUP_data.yaml --img 320 --weights runs/train/2_yolov7_tiny_seed_0/weights/epoch_083.pt --task test --name exp_debug

Namespace(weights=['runs/train/2_yolov7_tiny_seed_0/weights/epoch_083.pt'], data='data/DUP_data.yaml', batch_size=32, img_size=320, conf_thres=0.001, iou_thres=0.65, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp_debug', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 5c462a20 torch 2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4080, 15935.1875MB)

Fusing layers... 
IDetect.fuse
Model Summary: 208 layers, 6013008 parameters, 0 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/home/hj/anaconda3/envs/hj/lib/python3.13/site-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
test: Scannin

In [10]:
import subprocess
import numpy as np
import os

# --- Configuration ---
weights = [
    "runs/train/2_yolov7_tiny_seed_0/weights/epoch_083.pt",
    "runs/train/2_yolov7_tiny_seed_1/weights/epoch_083.pt",
    "runs/train/2_yolov7_tiny_seed_2/weights/epoch_083.pt",
    "runs/train/2_yolov7_tiny_seed_3/weights/epoch_083.pt",
    "runs/train/2_yolov7_tiny_seed_4/weights/epoch_083.pt"
]

# Path confirmed from your successful manual run
data = "data/DUP_data.yaml" 
img_size = 320

# --- Storage for results ---
map_50_list = []
map_50_95_list = []

print(f"Starting evaluation for {len(weights)} models...\n")

for w in weights:
    if not os.path.exists(w):
        print(f"Warning: Weight file not found: {w}")
        continue

    # Construct command (identical to your successful manual run)
    cmd = [
        "python", "-u", "test.py",
        "--data", data,
        "--img", str(img_size),
        "--device", "0",
        "--weights", w,
        "--task", "test",
        "--name", "exp_test_seeds"
    ]
    
    print(f"Testing: {w}")
    try:
        # Capture output
        result = subprocess.run(cmd, capture_output=True, text=True)
        full_output = result.stdout + "\n" + result.stderr
        
        found_stats = False
        
        # Parse the output line by line
        for line in full_output.split('\n'):
            # finding the 'all' row which contains the averages
            if line.strip().startswith('all'):
                parts = line.split()
                # Columns: 0:all, 1:imgs, 2:labels, 3:P, 4:R, 5:mAP@.5, 6:mAP@.5:.95
                if len(parts) >= 7:
                    try:
                        map50 = float(parts[5])
                        map5095 = float(parts[6])
                        
                        map_50_list.append(map50)
                        map_50_95_list.append(map5095)
                        
                        print(f"  -> mAP@0.5: {map50:.4f} | mAP@0.5:0.95: {map5095:.4f}")
                        found_stats = True
                        break
                    except ValueError:
                        continue 

        if not found_stats:
            print(f"  -> [ERROR] Could not find stats. Check if weights exist or paths are correct.")

    except Exception as e:
        print(f"  -> Python execution error: {e}")

# --- Calculate Statistics ---
if map_50_list:
    map50_mean = np.mean(map_50_list)
    map50_std = np.std(map_50_list)
    
    map5095_mean = np.mean(map_50_95_list)
    map5095_std = np.std(map_50_95_list)

    print("\n" + "="*50)
    print(f"FINAL RESULTS (Over {len(map_50_list)} seeds)")
    print("-" * 50)
    print(f"mAP@0.5      : {map50_mean:.4f} ± {map50_std:.4f}")
    print(f"mAP@0.5:0.95 : {map5095_mean:.4f} ± {map5095_std:.4f}")
    print("="*50)
else:
    print("\nNo results collected.")

Starting evaluation for 5 models...

Testing: runs/train/2_yolov7_tiny_seed_0/weights/epoch_083.pt
  -> mAP@0.5: 0.9740 | mAP@0.5:0.95: 0.7950
Testing: runs/train/2_yolov7_tiny_seed_1/weights/epoch_083.pt
  -> mAP@0.5: 0.9750 | mAP@0.5:0.95: 0.8200
Testing: runs/train/2_yolov7_tiny_seed_2/weights/epoch_083.pt
  -> mAP@0.5: 0.9740 | mAP@0.5:0.95: 0.7970
Testing: runs/train/2_yolov7_tiny_seed_3/weights/epoch_083.pt
  -> mAP@0.5: 0.9750 | mAP@0.5:0.95: 0.8080
Testing: runs/train/2_yolov7_tiny_seed_4/weights/epoch_083.pt
  -> mAP@0.5: 0.9810 | mAP@0.5:0.95: 0.8160

FINAL RESULTS (Over 5 seeds)
--------------------------------------------------
mAP@0.5      : 0.9758 ± 0.0026
mAP@0.5:0.95 : 0.8072 ± 0.0099


In [2]:
import pandas as pd

# 1. Define the correct headers for YOLOv7 results
headers = [
    "Epoch", "GPU_Mem", 
    "Train_Box", "Train_Obj", "Train_Cls", "Train_Total", 
    "Labels", "Img_Size", 
    "Precision", "Recall", "mAP_0.5", "mAP_0.5_0.95", 
    "Val_Box", "Val_Obj", "Val_Cls"
]

# 2. Read the whitespace-separated text file
# 'sep=r"\s+"' tells pandas to treat any amount of whitespace as a separator
df = pd.read_csv('runs/train/1_0_balanced_train_val_yolov7_tiny_seed0/results.txt', sep=r"\s+", names=headers, header=None)

# 3. Save to CSV
output_file = 'runs/train/1_0_balanced_train_val_yolov7_tiny_seed0/results.csv'
df.to_csv(output_file, index=False)

print(f"Successfully converted 'results.txt' to '{output_file}' with {len(df)} rows.")
print(df.head()) # Show the first few rows to verify

Successfully converted 'results.txt' to 'runs/train/1_0_balanced_train_val_yolov7_tiny_seed0/results.csv' with 100 rows.
  Epoch GPU_Mem  Train_Box  Train_Obj  Train_Cls  Train_Total  Labels  \
0  0/99   3.24G    0.07181    0.01210    0.03643      0.12030      13   
1  1/99   3.89G    0.06186    0.01126    0.03194      0.10510       5   
2  2/99   3.89G    0.05402    0.01094    0.02821      0.09316       6   
3  3/99   3.89G    0.04725    0.01078    0.02431      0.08234       8   
4  4/99   3.89G    0.04125    0.01034    0.01943      0.07102       3   

   Img_Size  Precision  Recall  mAP_0.5  mAP_0.5_0.95  Val_Box  Val_Obj  \
0       320     0.1762  0.3371   0.1182       0.03528  0.09434  0.01849   
1       320     0.1868  0.5422   0.2359       0.04930  0.09568  0.01530   
2       320     0.2623  0.5436   0.3158       0.07795  0.09343  0.01872   
3       320     0.4524  0.7047   0.6138       0.17580  0.07914  0.01826   
4       320     0.7778  0.7718   0.8094       0.36830  0.07095  0

In [3]:
# seed0: 0.9745
# seed1: 0.9776
# seed2: 0.9711
# seed3: 0.975
# seed4: 0.9759
# 0.9761, 0.9791, 0.9728, 0.9747, 0.9750

import numpy as np

# Your data
scores = [0.9745, 0.9776, 0.9711, 0.975, 0.9759]
# scores = [0.9761, 0.9791, 0.9728, 0.9747, 0.9750]
# Calculate Mean
mean_val = np.mean(scores)

# Calculate Standard Deviation
# ddof=1 calculates Sample Standard Deviation (standard for scientific papers)
# ddof=0 calculates Population Standard Deviation
std_val = np.std(scores, ddof=1) 

print(f"Mean: {mean_val:.5f}")
print(f"Std:  {std_val:.5f}")


Mean: 0.97482
Std:  0.00239
